In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from datetime import datetime

from tqdm import tqdm
from IPython.display import display

In [2]:
demog = pd.read_csv("DEMOGRAPHIC_DATA_TABLE.csv", sep = ";")
fin = pd.read_csv("FINANCIAL_TABLE.csv", sep = ";")
riskLimit = pd.read_csv("RISK_LIMIT_TABLE.csv", sep = ";")

display(demog.head())
display(demog.info())
display(fin.head())
display(fin.info())
display(riskLimit.head())
display(riskLimit.info())

,Log,Customer_num,Establishment_Date,Number_of_Emp,Sector,Region
0,14.06.2017 16:07,RATI9590GZD,26.01.2001,8.0,RETAIL-WHOLESALE,Marmara Region
1,20.02.2015 11:58,RATI2539VHR,24.02.1994,21.0,MANUFACTURING,Central Anatolia Region
2,07.12.2010 11:21,RATI4481GNN,25.01.1996,7.0,RETAIL-WHOLESALE,Mediterranean Region
3,02.10.2012 11:19,RATI4948THA,07.04.2004,34.0,MANUFACTURING,Southeastern Anatolia Region
4,27.05.2013 17:50,RATI8841WYZ,24.04.2006,15.0,SERVICES,Aegean Region


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11357 entries, 0 to 11356
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Log                 11357 non-null  object 
 1   Customer_num        11357 non-null  object 
 2   Establishment_Date  11357 non-null  object 
 3   Number_of_Emp       10771 non-null  float64
 4   Sector              11357 non-null  object 
 5   Region              10118 non-null  object 
dtypes: float64(1), object(5)
memory usage: 532.5+ KB


None

,FINANCIAL_YEAR,FINANCIAL_MONTH,Customer_num,Profit,Sales,Unnamed: 5
0,2017,12,RATI9590GZD,NaN,3,NaN
1,2015,12,RATI2539VHR,32615.0,0,NaN
2,2010,12,RATI4481GNN,282834.0,0,NaN
3,2012,12,RATI4948THA,35597.0,1,NaN
4,2013,12,RATI8841WYZ,134259.0,0,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11357 entries, 0 to 11356
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   FINANCIAL_YEAR   11357 non-null  int64  
 1   FINANCIAL_MONTH  11357 non-null  int64  
 2   Customer_num     11357 non-null  object 
 3   Profit           8526 non-null   float64
 4   Sales            11357 non-null  int64  
 5   Unnamed: 5       0 non-null      float64
dtypes: float64(2), int64(3), object(1)
memory usage: 532.5+ KB


None

,YEAR_MONTH,Customer_num,Total_Risk,Total_Limit
0,201012,RATI1023PGN,241927,404371
1,201012,RATI1040HVG,206680,251243
2,201012,RATI1077KXC,21314,1808901
3,201012,RATI1093DMU,22017,87604
4,201012,RATI1097YZH,7073447,29640487


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50724 entries, 0 to 50723
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   YEAR_MONTH     50724 non-null  int64 
 1   Customer_num   50724 non-null  object
 2   Total_Risk     50724 non-null  int64 
 3   Total_Limit    50724 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 1.5+ MB


None

In [3]:
print(f"There are {demog['Customer_num '].nunique()} customers in the demographic dataset.")
print(f"There are {fin['Customer_num '].nunique()} customers in the financial dataset.")
print(f"There are {riskLimit['Customer_num '].nunique()} customers in the limit-risk dataset.")

There are 4492 customers in the demographic dataset.
There are 4492 customers in the financial dataset.
There are 4450 customers in the limit-risk dataset.


## Initial Observations

For the demographic data table:

- There are null values present in both the 'Number of Employees' and 'Region' fields. A potential approach could be to fill these null values based on the 'Sector' field, provided there is a meaningful relationship between these fields.
- The 'Log' field appears to denote the timestamp of data collection. Divergent time periods across the data might lead to misleading results.
- The 'Establishment Date' field will be used to engineer a new feature representing the company's experience in its sector.

For the financial table:

- There are missing values in the 'Profit' field, specifically for companies where 'Sales' data is not available. I will investigate this further for confirmation.
- The 'Unnamed: 5' column will be dropped due to it being entirely void of data.

For the limit-risk table:

- This dataset is significantly larger compared to the other two, which leads me to infer that multiple periods of limit-risk data are available for each customer.
- I also plan on creating a feature representing the ratio of 'Risk' to 'Limit' for utilization in my predictive model.

## Inconsistencies

- There are inconsistencies in the temporal aspects of the data. For instance, while the 'Log' field in the demographic data seems to reflect varying time periods, financial data, specifically, is mostly recorded for the end of each year (December). Combining data with different time stamps might lead to skewed analyses.
- Furthermore, the demographic data reflects changes at different points within a year, while the financial data provides only a period-end snapshot. These discrepancies can lead to difficulties in accurate data merging and comparison.
- It's also important to consider that characteristics of companies such as the number of employees, sector, or region can change over time. These changes should ideally be reflected in the financial data, leading to potential discrepancies if they aren't.

In [4]:
demog.columns

Index(['Log', 'Customer_num ', 'Establishment_Date', 'Number_of_Emp', 'Sector',
       'Region'],
      dtype='object')

In [5]:
demog.head()

,Log,Customer_num,Establishment_Date,Number_of_Emp,Sector,Region
0,14.06.2017 16:07,RATI9590GZD,26.01.2001,8.0,RETAIL-WHOLESALE,Marmara Region
1,20.02.2015 11:58,RATI2539VHR,24.02.1994,21.0,MANUFACTURING,Central Anatolia Region
2,07.12.2010 11:21,RATI4481GNN,25.01.1996,7.0,RETAIL-WHOLESALE,Mediterranean Region
3,02.10.2012 11:19,RATI4948THA,07.04.2004,34.0,MANUFACTURING,Southeastern Anatolia Region
4,27.05.2013 17:50,RATI8841WYZ,24.04.2006,15.0,SERVICES,Aegean Region


In [6]:
demog.describe()

,Number_of_Emp
count,10771.000000
mean,16.611642
std,84.039303
min,1.000000
25%,4.000000
50%,7.000000
75%,16.000000
max,3333.000000


In [7]:
demog.Sector.value_counts()

Sector
RETAIL-WHOLESALE    5752
MANUFACTURING       2897
OTHERS              1798
SERVICES             910
Name: count, dtype: int64

In [8]:
demog.Region.value_counts()

Region
Marmara Region                  5166
Central Anatolia Region         1540
Aegean Region                   1212
Mediterranean Region            1129
Black Sea Region                 477
Southeastern Anatolia Region     398
Eastern Anatolia Region          196
Name: count, dtype: int64

There are duplicate rows in the demographic data. I want to identify the source of these duplicates.

In [9]:
column_list = ["Log", "Establishment_Date", "Sector", "Number_of_Emp", "Region"]

print(f"Number of rows in the original data: {demog.shape[0]}")

for i in range(1, len(column_list) + 1):
    demog_alter = demog.drop(column_list[:i], axis = 1).drop_duplicates()
    difference = demog.shape[0] - demog_alter.shape[0]
    dropped_columns = ", ".join(column_list[:i])
    print(f"\nWhen we drop '{dropped_columns}', number of rows left: {demog_alter.shape[0]}")
    print(f"Difference in the number of rows: {difference}")


Number of rows in the original data: 11357

When we drop 'Log', number of rows left: 11151
Difference in the number of rows: 206

When we drop 'Log, Establishment_Date', number of rows left: 7622
Difference in the number of rows: 3735

When we drop 'Log, Establishment_Date, Sector', number of rows left: 7622
Difference in the number of rows: 3735

When we drop 'Log, Establishment_Date, Sector, Number_of_Emp', number of rows left: 4685
Difference in the number of rows: 6672

When we drop 'Log, Establishment_Date, Sector, Number_of_Emp, Region', number of rows left: 4492
Difference in the number of rows: 6865


## Observations on the Demographic Data

- There are 206 rows that are identical except for the 'Log' column.
- When I drop the "Establishment_Date", the number of rows decreases. I presume there may be some errors in the dataset. New rows might have been added to correct previous errors. At this point, **I will assume that** the newer rows are more accurate."
- The 'Number_of_Emp' appears to vary for the same customer across different rows as well. There are different periods available in the financial dataset for different customers. I will use the number of employees of the related period.
- Dropping 'Sector' didn't make any difference. There are no companies operating in more than one sector.
- Dropping 'Region' changed the number of rows. It appears there are some companies operating in more than one region.

In [10]:
demog.columns = ["log", "customer_num", "establishment_date", "number_of_emp", "sector", "region"]
demog["log"] = pd.to_datetime(demog["log"].str.slice(0,10), dayfirst=True)
demog["quarter"] = demog['log'].dt.to_period("Q")
demog["establishment_date"] = pd.to_datetime(demog.establishment_date, dayfirst=True).dt.to_period("D")
demog = demog[["log", "customer_num", "quarter", "establishment_date", "number_of_emp", "sector", "region"]]
demog.head()

,log,customer_num,quarter,establishment_date,number_of_emp,sector,region
0,2017-06-14,RATI9590GZD,2017Q2,2001-01-26,8.0,RETAIL-WHOLESALE,Marmara Region
1,2015-02-20,RATI2539VHR,2015Q1,1994-02-24,21.0,MANUFACTURING,Central Anatolia Region
2,2010-12-07,RATI4481GNN,2010Q4,1996-01-25,7.0,RETAIL-WHOLESALE,Mediterranean Region
3,2012-10-02,RATI4948THA,2012Q4,2004-04-07,34.0,MANUFACTURING,Southeastern Anatolia Region
4,2013-05-27,RATI8841WYZ,2013Q2,2006-04-24,15.0,SERVICES,Aegean Region


I will start by changing the "Establishment Date" of the logs according to the neest log for each customer.

In [11]:
demog[demog.customer_num == "RATI9590GZD"]

,log,customer_num,quarter,establishment_date,number_of_emp,sector,region
0,2017-06-14,RATI9590GZD,2017Q2,2001-01-26,8.0,RETAIL-WHOLESALE,Marmara Region
8182,2016-06-04,RATI9590GZD,2016Q2,2001-02-03,8.0,RETAIL-WHOLESALE,Marmara Region
9891,2010-06-09,RATI9590GZD,2010Q2,2001-01-23,NaN,RETAIL-WHOLESALE,Marmara Region


In [12]:
for i in  tqdm(demog.customer_num.unique()):
    subdf = demog[demog.customer_num == i]
    date = subdf.log.max()
    est_date = subdf[(subdf.log == date)]["establishment_date"].values[0]
    idx = subdf.index
    demog.loc[idx, "establishment_date"] = est_date

100%|█████████████████████████████████████████████████████████████████████████████| 4492/4492 [00:07<00:00, 590.34it/s]


In [13]:
demog[demog.customer_num == "RATI9590GZD"]

,log,customer_num,quarter,establishment_date,number_of_emp,sector,region
0,2017-06-14,RATI9590GZD,2017Q2,2001-01-26,8.0,RETAIL-WHOLESALE,Marmara Region
8182,2016-06-04,RATI9590GZD,2016Q2,2001-01-26,8.0,RETAIL-WHOLESALE,Marmara Region
9891,2010-06-09,RATI9590GZD,2010Q2,2001-01-26,NaN,RETAIL-WHOLESALE,Marmara Region


Now let's chek for the companies operating in more than one region.

In [14]:
demog_alter = demog.drop(["log", "establishment_date", "number_of_emp", "sector", "quarter"], axis = 1).drop_duplicates()
doubleRegion = demog_alter.customer_num.value_counts()[demog_alter.customer_num.value_counts() == 2].keys()

In [15]:
for i in demog.index:
    if demog.loc[i, "customer_num"] in doubleRegion:
        if demog_alter[demog_alter.customer_num == demog.loc[i, "customer_num"]]["region"].isna().sum() <=0:
            print(f"{demog.loc[i, 'customer_num']} operates in more than more regions.")

RATI5542CKE operates in more than more regions.
RATI5542CKE operates in more than more regions.
RATI5542CKE operates in more than more regions.
RATI5542CKE operates in more than more regions.


In [16]:
demog[demog.customer_num == "RATI5542CKE"]

,log,customer_num,quarter,establishment_date,number_of_emp,sector,region
2710,2011-04-01,RATI5542CKE,2011Q2,2009-09-19,15.0,SERVICES,Black Sea Region
2725,2015-03-28,RATI5542CKE,2015Q1,2009-09-19,15.0,SERVICES,Central Anatolia Region
3503,2013-07-13,RATI5542CKE,2013Q3,2009-09-19,14.0,SERVICES,Black Sea Region
8024,2012-02-14,RATI5542CKE,2012Q1,2009-09-19,16.0,SERVICES,Black Sea Region


There is only one customer who operates in more than one region. This customer may have moved to the Central Anatolia Region, or this change might be a result of an error.

The rest of the duplicated rows are the result of null values. I will replace these null values using the existing information.

In [17]:
demog.isna().sum()

log                      0
customer_num             0
quarter                  0
establishment_date       0
number_of_emp          586
sector                   0
region                1239
dtype: int64

In [18]:
demog[demog.customer_num == "RATI9120QUZ"]

,log,customer_num,quarter,establishment_date,number_of_emp,sector,region
33,2010-03-04,RATI9120QUZ,2010Q1,1996-03-10,6.0,RETAIL-WHOLESALE,NaN
682,2011-05-19,RATI9120QUZ,2011Q2,1996-03-10,6.0,RETAIL-WHOLESALE,NaN
1605,2016-01-19,RATI9120QUZ,2016Q1,1996-03-10,8.0,RETAIL-WHOLESALE,Aegean Region
2650,2015-07-16,RATI9120QUZ,2015Q3,1996-03-10,8.0,RETAIL-WHOLESALE,Aegean Region
6488,2013-06-19,RATI9120QUZ,2013Q2,1996-03-10,6.0,RETAIL-WHOLESALE,NaN
9715,2012-06-12,RATI9120QUZ,2012Q2,1996-03-10,6.0,RETAIL-WHOLESALE,NaN


In [19]:
demog.region.value_counts()

region
Marmara Region                  5166
Central Anatolia Region         1540
Aegean Region                   1212
Mediterranean Region            1129
Black Sea Region                 477
Southeastern Anatolia Region     398
Eastern Anatolia Region          196
Name: count, dtype: int64

In [20]:
regionNullIdx = demog[demog.region.isna()].index
for i in tqdm(regionNullIdx):
    cus = demog.loc[i, "customer_num"]
    subDf = demog[demog.customer_num == cus].dropna()
    if len(subDf):
        reg = subDf.iloc[0, 6]
        demog.loc[i, "region"] = reg
demog.info()

100%|█████████████████████████████████████████████████████████████████████████████| 1239/1239 [00:01<00:00, 683.99it/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11357 entries, 0 to 11356
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   log                 11357 non-null  datetime64[ns]
 1   customer_num        11357 non-null  object        
 2   quarter             11357 non-null  period[Q-DEC] 
 3   establishment_date  11357 non-null  period[D]     
 4   number_of_emp       10771 non-null  float64       
 5   sector              11357 non-null  object        
 6   region              10514 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(3), period[D](1), period[Q-DEC](1)
memory usage: 621.2+ KB


In [21]:
demog.isna().sum()

log                     0
customer_num            0
quarter                 0
establishment_date      0
number_of_emp         586
sector                  0
region                843
dtype: int64

In [22]:
demog.region.value_counts()

region
Marmara Region                  5396
Central Anatolia Region         1612
Aegean Region                   1251
Mediterranean Region            1155
Black Sea Region                 492
Southeastern Anatolia Region     406
Eastern Anatolia Region          202
Name: count, dtype: int64

In [23]:
demog[demog.customer_num == "RATI9120QUZ"]

,log,customer_num,quarter,establishment_date,number_of_emp,sector,region
33,2010-03-04,RATI9120QUZ,2010Q1,1996-03-10,6.0,RETAIL-WHOLESALE,Aegean Region
682,2011-05-19,RATI9120QUZ,2011Q2,1996-03-10,6.0,RETAIL-WHOLESALE,Aegean Region
1605,2016-01-19,RATI9120QUZ,2016Q1,1996-03-10,8.0,RETAIL-WHOLESALE,Aegean Region
2650,2015-07-16,RATI9120QUZ,2015Q3,1996-03-10,8.0,RETAIL-WHOLESALE,Aegean Region
6488,2013-06-19,RATI9120QUZ,2013Q2,1996-03-10,6.0,RETAIL-WHOLESALE,Aegean Region
9715,2012-06-12,RATI9120QUZ,2012Q2,1996-03-10,6.0,RETAIL-WHOLESALE,Aegean Region


I will try to fill the null values in "number of employees" column using the existing data too. I have observed differing number of employees for some customers for different time periods. I will use the mean employee number of different periods to fil the null values.

In [24]:
demog[demog.customer_num == "RATI4576RCI"]

,log,customer_num,quarter,establishment_date,number_of_emp,sector,region
1197,2010-08-03,RATI4576RCI,2010Q3,1995-07-12,NaN,RETAIL-WHOLESALE,Central Anatolia Region
2186,2013-06-26,RATI4576RCI,2013Q2,1995-07-12,5.0,RETAIL-WHOLESALE,Central Anatolia Region
2966,2012-07-17,RATI4576RCI,2012Q3,1995-07-12,5.0,RETAIL-WHOLESALE,Central Anatolia Region
4748,2016-02-14,RATI4576RCI,2016Q1,1995-07-12,7.0,RETAIL-WHOLESALE,Central Anatolia Region
7166,2015-03-23,RATI4576RCI,2015Q1,1995-07-12,7.0,RETAIL-WHOLESALE,Central Anatolia Region
8563,2017-03-11,RATI4576RCI,2017Q1,1995-07-12,7.0,RETAIL-WHOLESALE,Central Anatolia Region
10509,2011-06-16,RATI4576RCI,2011Q2,1995-07-12,8.0,RETAIL-WHOLESALE,Central Anatolia Region
11176,2014-05-27,RATI4576RCI,2014Q2,1995-07-12,7.0,RETAIL-WHOLESALE,Central Anatolia Region


In [25]:
nullEmpNrCus = demog[demog.number_of_emp.isna()].customer_num.unique()
for i in tqdm(nullEmpNrCus):
    subDf = demog[demog.customer_num == i]
    val = subDf.number_of_emp.dropna().values
    if len(val):
        empNr = round(np.mean(val))
        idx = subDf[subDf.number_of_emp.isna()].index
        for j in idx:
            demog.loc[j, "number_of_emp"] = empNr
demog.info()

100%|███████████████████████████████████████████████████████████████████████████████| 571/571 [00:00<00:00, 775.60it/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11357 entries, 0 to 11356
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   log                 11357 non-null  datetime64[ns]
 1   customer_num        11357 non-null  object        
 2   quarter             11357 non-null  period[Q-DEC] 
 3   establishment_date  11357 non-null  period[D]     
 4   number_of_emp       11182 non-null  float64       
 5   sector              11357 non-null  object        
 6   region              10514 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(3), period[D](1), period[Q-DEC](1)
memory usage: 621.2+ KB


In [26]:
demog[demog.customer_num == "RATI4576RCI"]

,log,customer_num,quarter,establishment_date,number_of_emp,sector,region
1197,2010-08-03,RATI4576RCI,2010Q3,1995-07-12,7.0,RETAIL-WHOLESALE,Central Anatolia Region
2186,2013-06-26,RATI4576RCI,2013Q2,1995-07-12,5.0,RETAIL-WHOLESALE,Central Anatolia Region
2966,2012-07-17,RATI4576RCI,2012Q3,1995-07-12,5.0,RETAIL-WHOLESALE,Central Anatolia Region
4748,2016-02-14,RATI4576RCI,2016Q1,1995-07-12,7.0,RETAIL-WHOLESALE,Central Anatolia Region
7166,2015-03-23,RATI4576RCI,2015Q1,1995-07-12,7.0,RETAIL-WHOLESALE,Central Anatolia Region
8563,2017-03-11,RATI4576RCI,2017Q1,1995-07-12,7.0,RETAIL-WHOLESALE,Central Anatolia Region
10509,2011-06-16,RATI4576RCI,2011Q2,1995-07-12,8.0,RETAIL-WHOLESALE,Central Anatolia Region
11176,2014-05-27,RATI4576RCI,2014Q2,1995-07-12,7.0,RETAIL-WHOLESALE,Central Anatolia Region


In [27]:
demog.isna().sum()

log                     0
customer_num            0
quarter                 0
establishment_date      0
number_of_emp         175
sector                  0
region                843
dtype: int64

## Limit-Risk Data

In [28]:
riskLimit.isna().sum()

YEAR_MONTH       0
Customer_num     0
Total_Risk       0
Total_Limit      0
dtype: int64

In [29]:
riskLimit.head()

,YEAR_MONTH,Customer_num,Total_Risk,Total_Limit
0,201012,RATI1023PGN,241927,404371
1,201012,RATI1040HVG,206680,251243
2,201012,RATI1077KXC,21314,1808901
3,201012,RATI1093DMU,22017,87604
4,201012,RATI1097YZH,7073447,29640487


In [30]:
riskLimit.columns

Index(['YEAR_MONTH', 'Customer_num ', 'Total_Risk ', 'Total_Limit'], dtype='object')

In [31]:
riskLimit['year'] = riskLimit['YEAR_MONTH'] // 100
riskLimit['month'] = riskLimit['YEAR_MONTH'] % 100
riskLimit['date'] = pd.to_datetime(riskLimit['year'].astype(str) + riskLimit['month'].astype(str).str.zfill(2), format='%Y%m')
riskLimit["quarter"] = riskLimit['date'].dt.to_period("Q")
riskLimit = riskLimit[["Customer_num ", "quarter", "Total_Risk ", "Total_Limit"]]
riskLimit.columns = ["customer_num", "quarter", "total_risk", "total_limit"]
riskLimit.head()

,customer_num,quarter,total_risk,total_limit
0,RATI1023PGN,2010Q4,241927,404371
1,RATI1040HVG,2010Q4,206680,251243
2,RATI1077KXC,2010Q4,21314,1808901
3,RATI1093DMU,2010Q4,22017,87604
4,RATI1097YZH,2010Q4,7073447,29640487


## Financial Data

In [32]:
fin.head()

,FINANCIAL_YEAR,FINANCIAL_MONTH,Customer_num,Profit,Sales,Unnamed: 5
0,2017,12,RATI9590GZD,NaN,3,NaN
1,2015,12,RATI2539VHR,32615.0,0,NaN
2,2010,12,RATI4481GNN,282834.0,0,NaN
3,2012,12,RATI4948THA,35597.0,1,NaN
4,2013,12,RATI8841WYZ,134259.0,0,NaN


In [33]:
fin.isna().sum()

FINANCIAL_YEAR         0
FINANCIAL_MONTH        0
Customer_num           0
Profit              2831
Sales                  0
Unnamed: 5         11357
dtype: int64

In [34]:
fin.FINANCIAL_MONTH.value_counts()

FINANCIAL_MONTH
12    11331
9        18
6         5
3         2
8         1
Name: count, dtype: int64

Financial data seems to be quarterly. There is one instance where the month = 8. **I assume** it is an error. I will change it to 9.

In [35]:
fin[fin.FINANCIAL_MONTH == 8]

,FINANCIAL_YEAR,FINANCIAL_MONTH,Customer_num,Profit,Sales,Unnamed: 5
354,2015,8,RATI9716RNV,268128.0,0,NaN


In [36]:
fin[fin["Customer_num "] == "RATI9716RNV"]

,FINANCIAL_YEAR,FINANCIAL_MONTH,Customer_num,Profit,Sales,Unnamed: 5
354,2015,8,RATI9716RNV,268128.0,0,NaN
868,2017,12,RATI9716RNV,-7222.0,0,NaN
2549,2010,12,RATI9716RNV,155425.0,0,NaN
9124,2013,12,RATI9716RNV,-4542.0,0,NaN


In [37]:
fin['date'] = pd.to_datetime(fin['FINANCIAL_YEAR'].astype(str) + fin['FINANCIAL_MONTH'].astype(str).str.zfill(2), format='%Y%m')
fin["quarter"] = fin['date'].dt.to_period("Q")
fin = fin[["Customer_num ", "quarter", "Profit", "Sales"]]
fin.columns = ["customer_num", "quarter", "profit", "sales"]
fin.head()

,customer_num,quarter,profit,sales
0,RATI9590GZD,2017Q4,NaN,3
1,RATI2539VHR,2015Q4,32615.0,0
2,RATI4481GNN,2010Q4,282834.0,0
3,RATI4948THA,2012Q4,35597.0,1
4,RATI8841WYZ,2013Q4,134259.0,0


In [38]:
fin.loc[354]

customer_num    RATI9716RNV
quarter              2015Q3
profit             268128.0
sales                     0
Name: 354, dtype: object

In [39]:
fin.quarter.value_counts()

quarter
2017Q4    1762
2016Q4    1723
2015Q4    1600
2014Q4    1523
2013Q4    1391
2012Q4    1291
2011Q4    1124
2010Q4     917
2011Q3       4
2017Q3       4
2014Q3       3
2015Q3       3
2012Q2       2
2013Q3       2
2012Q3       1
2013Q2       1
2015Q1       1
2017Q1       1
2011Q2       1
2010Q3       1
2016Q3       1
2014Q2       1
Freq: Q-DEC, Name: count, dtype: int64

There are various periods available in the dataset but most of the data belongs to the end of the year. Let's check for the periods of the taraget data...

In [40]:
fin[fin.sales == 3].quarter.value_counts()

quarter
2017Q4    413
2015Q4    395
2016Q4    391
2014Q4    390
2013Q4    374
2012Q4    307
2011Q4    278
2010Q4    278
2014Q3      1
2011Q3      1
2010Q3      1
2013Q3      1
2017Q3      1
Freq: Q-DEC, Name: count, dtype: int64

Rows that sales = 3 are the target rows. After I am done with the data preperation, I will replace them with null.

In [41]:
fin.head()

,customer_num,quarter,profit,sales
0,RATI9590GZD,2017Q4,NaN,3
1,RATI2539VHR,2015Q4,32615.0,0
2,RATI4481GNN,2010Q4,282834.0,0
3,RATI4948THA,2012Q4,35597.0,1
4,RATI8841WYZ,2013Q4,134259.0,0


In [42]:
finDemog = pd.merge(fin, demog, on=['customer_num', 'quarter'], how='left')
finDemog.head()

,customer_num,quarter,profit,sales,log,establishment_date,number_of_emp,sector,region
0,RATI9590GZD,2017Q4,NaN,3,NaT,NaT,NaN,NaN,NaN
1,RATI2539VHR,2015Q4,32615.0,0,NaT,NaT,NaN,NaN,NaN
2,RATI4481GNN,2010Q4,282834.0,0,2010-12-07,1996-01-27,7.0,RETAIL-WHOLESALE,Mediterranean Region
3,RATI4948THA,2012Q4,35597.0,1,2012-10-02,2004-04-07,34.0,MANUFACTURING,Southeastern Anatolia Region
4,RATI8841WYZ,2013Q4,134259.0,0,NaT,NaT,NaN,NaN,NaN


In [43]:
finDemog.isna().sum()

customer_num             0
quarter                  0
profit                2833
sales                    0
log                   8431
establishment_date    8431
number_of_emp         8472
sector                8431
region                8651
dtype: int64

Demographic data periods are not compatible with the financial data. To fill the NaN rows, I will use the nearest available previous period. If no previous period is available, I will use the nearest future period.

In [44]:
for i in tqdm(finDemog[finDemog.establishment_date.isna()].index):
    customer = finDemog.loc[i, "customer_num"]
    q = finDemog.loc[i, "quarter"]
    prev = demog[(demog.customer_num == customer) & (demog.quarter < q)]
    if len(prev):
        q_prev = prev.quarter.max()
        arr1 = fin[(fin.customer_num == customer) & (fin.quarter == q)].iloc[0]
        arr2 = prev[prev.quarter == q_prev].iloc[0][["log","establishment_date","number_of_emp","sector","region"]]
        row = np.concatenate((arr1, arr2))
        finDemog.loc[i] = row
    else:
        future = demog[(demog.customer_num == customer) & (demog.quarter > q)]
        if len(future):
            q_future = future.quarter.min()
            arr1 = fin[(fin.customer_num == customer) & (fin.quarter == q)].iloc[0]
            arr2 = future[future.quarter == q_future].iloc[0][["log","establishment_date","number_of_emp","sector","region"]]
            row = np.concatenate((arr1, arr2))
            finDemog.loc[i] = row

100%|█████████████████████████████████████████████████████████████████████████████| 8431/8431 [00:31<00:00, 271.53it/s]


In [45]:
finDemog.isna().sum()

customer_num             0
quarter                  0
profit                2833
sales                    0
log                      0
establishment_date       0
number_of_emp          175
sector                   0
region                 843
dtype: int64

In [46]:
finDemog.head()

,customer_num,quarter,profit,sales,log,establishment_date,number_of_emp,sector,region
0,RATI9590GZD,2017Q4,NaN,3,2017-06-14,2001-01-26,8.0,RETAIL-WHOLESALE,Marmara Region
1,RATI2539VHR,2015Q4,32615.0,0,2015-02-20,1994-03-08,21.0,MANUFACTURING,Central Anatolia Region
2,RATI4481GNN,2010Q4,282834.0,0,2010-12-07,1996-01-27,7.0,RETAIL-WHOLESALE,Mediterranean Region
3,RATI4948THA,2012Q4,35597.0,1,2012-10-02,2004-04-07,34.0,MANUFACTURING,Southeastern Anatolia Region
4,RATI8841WYZ,2013Q4,134259.0,0,2013-05-27,2006-04-24,15.0,SERVICES,Aegean Region


In [47]:
riskLimit.describe()

,total_risk,total_limit
count,5.072400e+04,5.072400e+04
mean,1.322765e+06,2.561446e+06
std,5.080110e+07,2.053292e+07
min,-4.960000e+02,2.296000e+03
25%,1.183225e+05,5.084618e+05
50%,3.847560e+05,1.268974e+06
75%,9.829458e+05,2.892633e+06
max,5.857059e+09,2.148731e+09


Risk of a company shouldn't be smaller than 0. I will change these as 0.

In [48]:
errIdx = riskLimit[riskLimit.total_risk<0].index
riskLimit.loc[errIdx, "total_risk"] = 0
riskLimit.describe()

,total_risk,total_limit
count,5.072400e+04,5.072400e+04
mean,1.322766e+06,2.561446e+06
std,5.080110e+07,2.053292e+07
min,0.000000e+00,2.296000e+03
25%,1.183225e+05,5.084618e+05
50%,3.847560e+05,1.268974e+06
75%,9.829458e+05,2.892633e+06
max,5.857059e+09,2.148731e+09


There might be instances where the credit risk exceeds credit limit bu they are extremely rare. To avoid complications, I will change these risks to be equal to limit.

In [49]:
errIdx = riskLimit[riskLimit.total_risk > riskLimit.total_limit].index
for i in errIdx:
    riskLimit.loc[i, "total_risk"] = riskLimit.loc[i, "total_limit"]
    
riskLimit.describe()

,total_risk,total_limit
count,5.072400e+04,5.072400e+04
mean,1.014875e+06,2.561446e+06
std,1.994116e+07,2.053292e+07
min,0.000000e+00,2.296000e+03
25%,1.181130e+05,5.084618e+05
50%,3.843645e+05,1.268974e+06
75%,9.811885e+05,2.892633e+06
max,2.148731e+09,2.148731e+09


In [50]:
riskLimit[riskLimit.total_risk > riskLimit.total_limit]

,customer_num,quarter,total_risk,total_limit


In [51]:
finalDf = pd.merge(finDemog, riskLimit, on=['customer_num', 'quarter'], how='left')
finalDf.head()

,customer_num,quarter,profit,sales,log,establishment_date,number_of_emp,sector,region,total_risk,total_limit
0,RATI9590GZD,2017Q4,NaN,3,2017-06-14,2001-01-26,8.0,RETAIL-WHOLESALE,Marmara Region,70614.0,7000362.0
1,RATI2539VHR,2015Q4,32615.0,0,2015-02-20,1994-03-08,21.0,MANUFACTURING,Central Anatolia Region,682662.0,2353551.0
2,RATI4481GNN,2010Q4,282834.0,0,2010-12-07,1996-01-27,7.0,RETAIL-WHOLESALE,Mediterranean Region,115745.0,592723.0
3,RATI4948THA,2012Q4,35597.0,1,2012-10-02,2004-04-07,34.0,MANUFACTURING,Southeastern Anatolia Region,39234.0,2470770.0
4,RATI8841WYZ,2013Q4,134259.0,0,2013-05-27,2006-04-24,15.0,SERVICES,Aegean Region,70894.0,505769.0


In [52]:
finalDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11361 entries, 0 to 11360
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   customer_num        11361 non-null  object        
 1   quarter             11361 non-null  period[Q-DEC] 
 2   profit              8528 non-null   float64       
 3   sales               11361 non-null  int64         
 4   log                 11361 non-null  datetime64[ns]
 5   establishment_date  11361 non-null  period[D]     
 6   number_of_emp       11186 non-null  float64       
 7   sector              11361 non-null  object        
 8   region              10518 non-null  object        
 9   total_risk          11244 non-null  float64       
 10  total_limit         11244 non-null  float64       
dtypes: datetime64[ns](1), float64(4), int64(1), object(3), period[D](1), period[Q-DEC](1)
memory usage: 976.5+ KB


In [53]:
finalDf[finalDf.profit.isna()].sales.value_counts()

sales
3    2833
Name: count, dtype: int64

All missing values in the profit column belong to the target class. Since it will be useless for the model, I will drop this column.

In [54]:
finalDf.drop("profit", axis = 1, inplace = True)

In the analysis, it was suggested that the change in the limit in recent months could also be significant. To verify this hypothesis, I will calculate the percentage change of both limit and risk over the most recent four periods. If the limit and risk data are unavailable for these periods, I will operate under the assumption that they have remained unchanged.

In [55]:
finalDf.describe()

,sales,log,number_of_emp,total_risk,total_limit
count,11361.000000,11361,11186.000000,1.124400e+04,1.124400e+04
mean,0.875187,2014-06-09 04:19:19.757063680,16.525121,1.005068e+06,2.533506e+06
min,0.000000,2010-01-01 00:00:00,1.000000,0.000000e+00,2.518000e+03
25%,0.000000,2012-08-14 00:00:00,4.000000,1.194502e+05,5.136748e+05
50%,0.000000,2014-08-21 00:00:00,8.000000,3.951700e+05,1.289093e+06
75%,1.000000,2016-05-17 00:00:00,16.000000,1.005277e+06,2.932201e+06
max,3.000000,2017-12-28 00:00:00,3333.000000,1.894825e+09,1.894825e+09
std,1.267102,NaN,82.545126,1.801998e+07,1.858756e+07


In [56]:
totalRiskT1 = []
totalRiskT2 = []
totalRiskT3 = []
totalRiskT4 = []
totalLimitT1 = []
totalLimitT2 = []
totalLimitT3 = []
totalLimitT4 = []

for i in tqdm(finalDf.index):
    cus = finalDf.loc[i, "customer_num"]
    q = finalDf.loc[i, "quarter"]
    
    subdf = riskLimit[(riskLimit.customer_num == cus)&(riskLimit.quarter == q-1)]
    if len(subdf):
        totalRiskT1.append(subdf.total_risk.values[0])
        totalLimitT1.append(subdf.total_limit.values[0])
    else:
        totalRiskT1.append(np.nan)
        totalLimitT1.append(np.nan)
    subdf = riskLimit[(riskLimit.customer_num == cus)&(riskLimit.quarter == q-2)]
    if len(subdf):
        totalRiskT2.append(subdf.total_risk.values[0])
        totalLimitT2.append(subdf.total_limit.values[0])
    else:
        totalRiskT2.append(np.nan)
        totalLimitT2.append(np.nan)
    subdf = riskLimit[(riskLimit.customer_num == cus)&(riskLimit.quarter == q-3)]
    if len(subdf):
        totalRiskT3.append(subdf.total_risk.values[0])
        totalLimitT3.append(subdf.total_limit.values[0])
    else:
        totalRiskT3.append(np.nan)
        totalLimitT3.append(np.nan)
    subdf = riskLimit[(riskLimit.customer_num == cus)&(riskLimit.quarter == q-4)]
    if len(subdf):
        totalRiskT4.append(subdf.total_risk.values[0])
        totalLimitT4.append(subdf.total_limit.values[0])
    else:
        totalRiskT4.append(np.nan)
        totalLimitT4.append(np.nan)

100%|████████████████████████████████████████████████████████████████████████████| 11361/11361 [02:01<00:00, 93.41it/s]


In [57]:
finalDf["totalRiskT1"] = totalRiskT1
finalDf["totalRiskT2"] = totalRiskT2
finalDf["totalRiskT3"] = totalRiskT3
finalDf["totalRiskT4"] = totalRiskT4
finalDf["totalLimitT1"] = totalLimitT1
finalDf["totalLimitT2"] = totalLimitT2
finalDf["totalLimitT3"] = totalLimitT3
finalDf["totalLimitT4"] = totalLimitT4

finalDf.head()    

,customer_num,quarter,sales,log,establishment_date,number_of_emp,sector,region,total_risk,total_limit,totalRiskT1,totalRiskT2,totalRiskT3,totalRiskT4,totalLimitT1,totalLimitT2,totalLimitT3,totalLimitT4
0,RATI9590GZD,2017Q4,3,2017-06-14,2001-01-26,8.0,RETAIL-WHOLESALE,Marmara Region,70614.0,7000362.0,77004.0,75002.0,67859.0,47342.0,6299583.0,6867406.0,6935723.0,5860498.0
1,RATI2539VHR,2015Q4,0,2015-02-20,1994-03-08,21.0,MANUFACTURING,Central Anatolia Region,682662.0,2353551.0,675909.0,709079.0,738072.0,683153.0,2188838.0,2298960.0,2137717.0,3035324.0
2,RATI4481GNN,2010Q4,0,2010-12-07,1996-01-27,7.0,RETAIL-WHOLESALE,Mediterranean Region,115745.0,592723.0,112913.0,105124.0,111205.0,116312.0,557378.0,507497.0,497358.0,522139.0
3,RATI4948THA,2012Q4,1,2012-10-02,2004-04-07,34.0,MANUFACTURING,Southeastern Anatolia Region,39234.0,2470770.0,39148.0,38039.0,37282.0,35821.0,2422098.0,2373239.0,2302184.0,2071846.0
4,RATI8841WYZ,2013Q4,0,2013-05-27,2006-04-24,15.0,SERVICES,Aegean Region,70894.0,505769.0,70337.0,63175.0,63554.0,80901.0,495645.0,511140.0,551499.0,556069.0


In [58]:
finalDf.isna().sum()

customer_num            0
quarter                 0
sales                   0
log                     0
establishment_date      0
number_of_emp         175
sector                  0
region                843
total_risk            117
total_limit           117
totalRiskT1           165
totalRiskT2           165
totalRiskT3           165
totalRiskT4           123
totalLimitT1          165
totalLimitT2          165
totalLimitT3          165
totalLimitT4          123
dtype: int64

In [59]:
idx = finalDf[finalDf.totalLimitT4.isna()].index
finalDf.loc[idx]

,customer_num,quarter,sales,log,establishment_date,number_of_emp,sector,region,total_risk,total_limit,totalRiskT1,totalRiskT2,totalRiskT3,totalRiskT4,totalLimitT1,totalLimitT2,totalLimitT3,totalLimitT4
24,RATI1744GHE,2012Q3,0,2012-11-26,1996-01-22,2.0,RETAIL-WHOLESALE,Marmara Region,189167.0,233749.0,192341.0,177049.0,177477.0,NaN,252023.0,251725.0,269762.0,NaN
49,RATI6245IHU,2010Q4,0,2010-12-05,2008-02-04,1.0,RETAIL-WHOLESALE,Marmara Region,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
118,RATI5303MNE,2013Q4,3,2013-01-20,2006-09-25,1.0,RETAIL-WHOLESALE,Central Anatolia Region,15011.0,166193.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
226,RATI6197ZQH,2011Q4,0,2011-12-13,2007-12-01,10.0,RETAIL-WHOLESALE,Marmara Region,244118.0,916966.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
274,RATI8462QWT,2011Q2,0,2011-06-12,2007-05-31,3.0,RETAIL-WHOLESALE,Mediterranean Region,2345443.0,4814281.0,2110903.0,2025846.0,NaN,NaN,4958706.0,4512247.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11112,RATI9892HXU,2010Q4,0,2010-04-13,2005-02-18,NaN,SERVICES,Aegean Region,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11139,RATI1443HSN,2011Q4,3,2011-12-01,1996-02-01,8.0,RETAIL-WHOLESALE,Marmara Region,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11182,RATI2031VPW,2015Q4,0,2015-12-15,1994-12-27,5.0,MANUFACTURING,Marmara Region,455746.0,1708089.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11264,RATI1560MFM,2012Q4,0,2012-01-14,2002-06-21,2.0,RETAIL-WHOLESALE,Marmara Region,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [60]:
limit_columns = ['total_limit', 'totalLimitT1', 'totalLimitT2', 'totalLimitT3', 'totalLimitT4']
risk_columns = ['total_risk', 'totalRiskT1', 'totalRiskT2', 'totalRiskT3', 'totalRiskT4']

for i in range(1, 5):
    finalDf[limit_columns[i]].fillna(finalDf[limit_columns[i-1]], inplace=True)
    finalDf[risk_columns[i]].fillna(finalDf[risk_columns[i-1]], inplace=True)
    
finalDf.loc[idx]

,customer_num,quarter,sales,log,establishment_date,number_of_emp,sector,region,total_risk,total_limit,totalRiskT1,totalRiskT2,totalRiskT3,totalRiskT4,totalLimitT1,totalLimitT2,totalLimitT3,totalLimitT4
24,RATI1744GHE,2012Q3,0,2012-11-26,1996-01-22,2.0,RETAIL-WHOLESALE,Marmara Region,189167.0,233749.0,192341.0,177049.0,177477.0,177477.0,252023.0,251725.0,269762.0,269762.0
49,RATI6245IHU,2010Q4,0,2010-12-05,2008-02-04,1.0,RETAIL-WHOLESALE,Marmara Region,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
118,RATI5303MNE,2013Q4,3,2013-01-20,2006-09-25,1.0,RETAIL-WHOLESALE,Central Anatolia Region,15011.0,166193.0,15011.0,15011.0,15011.0,15011.0,166193.0,166193.0,166193.0,166193.0
226,RATI6197ZQH,2011Q4,0,2011-12-13,2007-12-01,10.0,RETAIL-WHOLESALE,Marmara Region,244118.0,916966.0,244118.0,244118.0,244118.0,244118.0,916966.0,916966.0,916966.0,916966.0
274,RATI8462QWT,2011Q2,0,2011-06-12,2007-05-31,3.0,RETAIL-WHOLESALE,Mediterranean Region,2345443.0,4814281.0,2110903.0,2025846.0,2025846.0,2025846.0,4958706.0,4512247.0,4512247.0,4512247.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11112,RATI9892HXU,2010Q4,0,2010-04-13,2005-02-18,NaN,SERVICES,Aegean Region,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11139,RATI1443HSN,2011Q4,3,2011-12-01,1996-02-01,8.0,RETAIL-WHOLESALE,Marmara Region,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11182,RATI2031VPW,2015Q4,0,2015-12-15,1994-12-27,5.0,MANUFACTURING,Marmara Region,455746.0,1708089.0,455746.0,455746.0,455746.0,455746.0,1708089.0,1708089.0,1708089.0,1708089.0
11264,RATI1560MFM,2012Q4,0,2012-01-14,2002-06-21,2.0,RETAIL-WHOLESALE,Marmara Region,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
finalDf.isna().sum()

customer_num            0
quarter                 0
sales                   0
log                     0
establishment_date      0
number_of_emp         175
sector                  0
region                843
total_risk            117
total_limit           117
totalRiskT1           117
totalRiskT2           117
totalRiskT3           116
totalRiskT4            79
totalLimitT1          117
totalLimitT2          117
totalLimitT3          116
totalLimitT4           79
dtype: int64

I will create a column representing company age as days. I also want to crate a column including risk to limit ratio but I will create the column after imputing missing values in the limit and risk columns.

In [62]:
finalDf["companyAge"] = round((datetime(year = 2017, month = 12, day = 31) - finalDf["establishment_date"].dt.to_timestamp()).dt.days/365, 2
)
finalDf.head()

,customer_num,quarter,sales,log,establishment_date,number_of_emp,sector,region,total_risk,total_limit,totalRiskT1,totalRiskT2,totalRiskT3,totalRiskT4,totalLimitT1,totalLimitT2,totalLimitT3,totalLimitT4,companyAge
0,RATI9590GZD,2017Q4,3,2017-06-14,2001-01-26,8.0,RETAIL-WHOLESALE,Marmara Region,70614.0,7000362.0,77004.0,75002.0,67859.0,47342.0,6299583.0,6867406.0,6935723.0,5860498.0,16.94
1,RATI2539VHR,2015Q4,0,2015-02-20,1994-03-08,21.0,MANUFACTURING,Central Anatolia Region,682662.0,2353551.0,675909.0,709079.0,738072.0,683153.0,2188838.0,2298960.0,2137717.0,3035324.0,23.83
2,RATI4481GNN,2010Q4,0,2010-12-07,1996-01-27,7.0,RETAIL-WHOLESALE,Mediterranean Region,115745.0,592723.0,112913.0,105124.0,111205.0,116312.0,557378.0,507497.0,497358.0,522139.0,21.94
3,RATI4948THA,2012Q4,1,2012-10-02,2004-04-07,34.0,MANUFACTURING,Southeastern Anatolia Region,39234.0,2470770.0,39148.0,38039.0,37282.0,35821.0,2422098.0,2373239.0,2302184.0,2071846.0,13.74
4,RATI8841WYZ,2013Q4,0,2013-05-27,2006-04-24,15.0,SERVICES,Aegean Region,70894.0,505769.0,70337.0,63175.0,63554.0,80901.0,495645.0,511140.0,551499.0,556069.0,11.70


In the 'sales' column, I will replace the '3' values with 'NaN', as these are the values that we intend to predict.

In [63]:
finalDf["sales"].replace({3:np.nan}, inplace=True)
finalDf.head()

,customer_num,quarter,sales,log,establishment_date,number_of_emp,sector,region,total_risk,total_limit,totalRiskT1,totalRiskT2,totalRiskT3,totalRiskT4,totalLimitT1,totalLimitT2,totalLimitT3,totalLimitT4,companyAge
0,RATI9590GZD,2017Q4,NaN,2017-06-14,2001-01-26,8.0,RETAIL-WHOLESALE,Marmara Region,70614.0,7000362.0,77004.0,75002.0,67859.0,47342.0,6299583.0,6867406.0,6935723.0,5860498.0,16.94
1,RATI2539VHR,2015Q4,0.0,2015-02-20,1994-03-08,21.0,MANUFACTURING,Central Anatolia Region,682662.0,2353551.0,675909.0,709079.0,738072.0,683153.0,2188838.0,2298960.0,2137717.0,3035324.0,23.83
2,RATI4481GNN,2010Q4,0.0,2010-12-07,1996-01-27,7.0,RETAIL-WHOLESALE,Mediterranean Region,115745.0,592723.0,112913.0,105124.0,111205.0,116312.0,557378.0,507497.0,497358.0,522139.0,21.94
3,RATI4948THA,2012Q4,1.0,2012-10-02,2004-04-07,34.0,MANUFACTURING,Southeastern Anatolia Region,39234.0,2470770.0,39148.0,38039.0,37282.0,35821.0,2422098.0,2373239.0,2302184.0,2071846.0,13.74
4,RATI8841WYZ,2013Q4,0.0,2013-05-27,2006-04-24,15.0,SERVICES,Aegean Region,70894.0,505769.0,70337.0,63175.0,63554.0,80901.0,495645.0,511140.0,551499.0,556069.0,11.70


In [64]:
finalDf.isna().sum()

customer_num             0
quarter                  0
sales                 2833
log                      0
establishment_date       0
number_of_emp          175
sector                   0
region                 843
total_risk             117
total_limit            117
totalRiskT1            117
totalRiskT2            117
totalRiskT3            116
totalRiskT4             79
totalLimitT1           117
totalLimitT2           117
totalLimitT3           116
totalLimitT4            79
companyAge               0
dtype: int64

The aim of the study is to predict the **annual** sales of companies. Using data from the 1st, 2nd, and 3rd quarters to predict annual sales may decrease the model's performance. For this reason, I will exclude rows containing information from these periods.

In [65]:
finalDf = finalDf[finalDf.quarter.dt.quarter == 4]

In [66]:
finalDf.to_csv("preprocessed_data.csv", index = False)

## Notes for the Next Notebook

- In this notebook, I have merged demographic, financial, and limit-risk data for companies. During this process, I backfilled some of the NaN values using existing information from the customers. The remaining NaN values will be imputed within a pipeline in the next notebook to avoid data leakage.
- The demographic data contained different establishment dates for each company across different rows. I have updated the establishment date for each customer to the most recent date in the logs, **assuming** these are more accurate.
- To create the final data frame, I generated a "quarter" column in each dataset and used this information as a key.
- I have created a 'company age' column, **assuming** that the most recent date in the dataset is December 31, 2017.
- I have also generated additional columns representing the total limit and risk for the previous quarters. Later, I will add columns representing the percentage changes in the limit and risk. However, to ensure accuracy in this process, I first need to impute any missing values in the limit and risk columns.
- I will perform this imputation within a pipeline to avoid data leakage from the test set to the training set.